In [1]:
import pandas as pd
import numpy as np
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold

In [2]:
train = pd.read_csv("../input/train_bankcard_info.csv")
test = pd.read_csv("../input/test_bankcard_info.csv")
train.head(),test.head(),train.shape,test.shape

(  bank_name  tail_num card_type        phone                  id
 0      中信银行      9753       储蓄卡  177****8427  585401206734589960
 1      农业银行      2174       储蓄卡  150****4551  587694980785508360
 2    中国交通银行      4832       信用卡  185****1989  602098357531316232
 3      北京银行      5697       储蓄卡  186****1634  567205103669350408
 4      中信银行      7703       储蓄卡  158****7655  577538085852549384,
   bank_name  tail_num card_type        phone                  id
 0      招商银行       649       信用卡  150****2130  618646533478289672
 1      农业银行      4977       储蓄卡  183****4259  622333276798849032
 2    中国交通银行      4086       信用卡  134****6777  618654112241684744
 3    中国光大银行      5395       储蓄卡  135****5444  633152330136555784
 4      工商银行      5763       储蓄卡  135****5444  633152330136555784,
 (315489, 5),
 (96149, 5))

In [3]:
def tranf(x):
    x = str(x)
    if x.startswith(('建设','ccb')):
        x= 'a'
    elif x.startswith(('工商','icbc')):
        x = 'b'
    elif x.startswith(('农业','abc')):
        x = 'c'
    elif x.startswith(('招商','cmb')):
        x = 'd' 
    elif x.startswith(('中国银行','boc')):
        x = 'e'
    elif x.startswith(('中信银行','ccb','citic')):
        x = 'f'
    elif x.startswith(('中国交通','bocom')):
        x = 'g'
    elif x.startswith(('中国光大','ceb')):
        x = 'h'
    elif x.startswith(('邮政储蓄','psbc','post')):
        x = 'i'
    elif x.startswith(('中国民生','cmbc')):
        x = 'j'
    elif x.startswith(('广发','gdb')):
        x = 'k'
    elif x.startswith(('平安','pab')):
        x = 'l'
    elif x.startswith(('兴业','cib')):
        x = 'm'
    elif x.startswith(('浦发','spdb')):
        x = 'n'
    elif x.startswith(('bcom')):
        x = 'nk'
    elif x.startswith(('华夏银行','hxb')):
        x = 'nk'
    else :
        x ='o'
    return x

def card(x):
    if x == "储蓄卡":
        return 1
    if x == "信用卡":
        return 2
    else:
        return 3
    
#'bank_name', 'tail_num', 'card_type', 'phone', 'id'], dtype='object')
# 订单号本程序模块化
# 看一下是否所有手机号一致
def groupDeal(group):

    bank_name_more_cate = group.bank_name.value_counts().sort_values(ascending = False).index[0]
    bank_name_less_cate = group.bank_name.value_counts().sort_values(ascending = True).index[0]
    bank_name_more_count = group.bank_name.value_counts().sort_values(ascending = False)[0]
    bank_name_less_count = group.bank_name.value_counts().sort_values(ascending = True)[0]
   
    card_type_more_cate = group.card_type.value_counts().sort_values(ascending = False).index[0]
    card_type_less_cate = group.card_type.value_counts().sort_values(ascending = True).index[0]
    card_type_more_count = group.card_type.value_counts().sort_values(ascending = False).iloc[0]
    card_type_less_count = group.card_type.value_counts().sort_values(ascending = True).iloc[0]
    
    phone_more_cate = group.phone.value_counts().sort_values(ascending = False).index[0]
    phone_less_cate = group.phone.value_counts().sort_values(ascending = True).index[0]
    phone_more_count = group.phone.value_counts().sort_values(ascending = False)[0]
    phone_less_count = group.phone.value_counts().sort_values(ascending = True).iloc[0]
    phone_num = group.phone.shape[0]
    #看一下所有的卡是否用一张手机号办的
    number = 0
    phoneSame = 0  #0 表示相同
    if phone_num > 1:
        for j in range(1,phone_num):
            if group["phone"].iloc[0] != group["phone"].iloc[j]:
                number = 5
                break
        if number == 5:
            phoneSame = 1
    #看一下有无信用卡
    number1 = 0
    havaCreditCard = 0  #0表示没有
    for j in range(0,phone_num):
        if group["card_type"].iloc[j] == 2:
            number1 = 5
            break
    if number1 == 5:
        havaCreditCard = 1
    # 尾号不一样
#     tail_num_max = group.tail_num.max()
#     tail_num_min = group.tail_num.min()
#     tail_num_mean = round(group.tail_num.mean())
# #     tail_num_std = group.tail_num.std()  #很多空值，不要
#     tail_num_maxCutmean = tail_num_max - tail_num_mean
#     tail_num_more_num = group.tail_num.value_counts().sort_values(ascending = False).index[0]
#     tail_num_less_num = group.tail_num.value_counts().sort_values(ascending = True).index[0]
#     tail_num_more_count = group.tail_num.value_counts().sort_values(ascending = False)[0]
#     tail_num_less_count = group.tail_num.value_counts().sort_values(ascending = True)[0]
#     tail_num_moreCutmean =  tail_num_more_num - tail_num_mean
#     tail_num_num = group.tail_num.shape[0]
    
    indexs = {
    "bank_name_more_cate":bank_name_more_cate,
    "bank_name_less_cate": bank_name_less_cate,
    "card_type_more_cate":card_type_more_cate,
    "card_type_less_cate": card_type_less_cate,
    "phone_more_cate":phone_more_cate,
    "phone_less_cate": phone_less_cate,
    "phoneSame":phoneSame,
    "havaCreditCard":havaCreditCard,   
        
    "bank_name_more_count":bank_name_more_count,
    "bank_name_less_count": bank_name_less_count,
    "card_type_more_count":card_type_more_count,
    "card_type_less_count": card_type_less_count,
    "phone_more_count":phone_more_count,
    "phone_less_count": phone_less_count,
    "phone_num": phone_num,
#     "tail_num_max": tail_num_max,
#     "tail_num_min": tail_num_min,
#     "tail_num_mean": tail_num_mean,
#     "tail_num_std": tail_num_std,
#     "tail_num_maxCutmean": tail_num_maxCutmean,
# #     "tail_num_more_num": tail_num_more_num,
#     "tail_num_less_num": tail_num_less_num,
#     "tail_num_more_count": tail_num_more_count,
#     "tail_num_less_count": tail_num_less_count,
#     "tail_num_moreCutmean": tail_num_moreCutmean,
#     "tail_num_num": tail_num_num
    }
    return pd.Series(data=[indexs[c] for c in indexs], index=[c for c in indexs])

In [4]:
train = pd.read_csv("../input/train_bankcard_info.csv")
test = pd.read_csv("../input/test_bankcard_info.csv")
'''
记得后面加一个特征：是否有10大银行信用卡，有说明信用还可以
tr = pd.Series(train.id.unique())
te = pd.Series(test.id.unique())
te.apply(lambda x: x in tr).unique()
训练集和测试集都是独立的，所以可以拼接起来，同一处理
'''
data = pd.concat([train,test],axis = 0,ignore_index = True)
data.bank_name.fillna(data.bank_name.value_counts().sort_values(ascending = False).index[0],inplace = True)
# print(data.shape)
# print(data.head())
# print(data.columns)
# print(data.bank_name.unique())
# data.tail_num.unique()


data.card_type = data.card_type.apply(lambda x: card(x)).astype("float")
data.bank_name = data.bank_name.apply(lambda x:tranf(x))
# data.head(),data.info()
# tmp1 = data.iloc[0:30,:].groupby(by=['id']).apply(groupDeal)
# print("djhasdfgsdhjghsdugioh")
# tmp1.shape
tmp1 = data.groupby(by=['id']).apply(groupDeal)


In [5]:
print(tmp1.shape)
tmp1["id"] = tmp1.index
tmp1.to_csv("../input/bank_tmp_group.csv",index = False)
tmp1

(167960, 15)


,bank_name_more_cate,bank_name_less_cate,card_type_more_cate,card_type_less_cate,phone_more_cate,phone_less_cate,phoneSame,havaCreditCard,bank_name_more_count,bank_name_less_count,card_type_more_count,card_type_less_count,phone_more_count,phone_less_count,phone_num,id
id,,,,,,,,,,,,,,,,
616266486771618056,b,b,1.0,1.0,151****2492,151****2492,0,0,1,1,1,1,1,1,1,616266486771618056
616411495579783176,a,b,1.0,2.0,159****3061,159****3061,0,1,1,1,2,1,3,3,3,616411495579783176
616483294195749128,d,g,2.0,1.0,135****1100,135****1100,0,1,1,1,2,1,3,3,3,616483294195749128
616547037910929672,f,l,1.0,1.0,189****4181,189****4181,0,0,1,1,2,2,2,2,2,616547037910929672
616550964970656008,e,b,1.0,1.0,159****2951,182****1869,1,0,1,1,2,2,1,1,2,616550964970656008
616657485251481864,a,g,1.0,1.0,152****2800,152****2800,0,0,2,1,6,6,6,6,6,616657485251481864
616758963400085768,a,a,2.0,2.0,132****3536,132****3536,0,1,1,1,1,1,1,1,1,616758963400085768
616772308744278024,c,c,1.0,1.0,187****1769,187****1769,0,0,1,1,1,1,1,1,1,616772308744278024
616777971096621320,e,e,1.0,1.0,183****2991,183****2991,0,0,1,1,1,1,1,1,1,616777971096621320


In [6]:
train = pd.read_csv("../dealinput/train1.csv")
test = pd.read_csv("../dealinput/test1.csv")
# train.head(),test.head(),train.shape,test.shape
mm = train[["id","target"]]
nn = pd.DataFrame(test.id)
trainnew = mm.merge(tmp1,how = "left",on = "id")
testnew = nn.merge(tmp1,how = "left",on = "id")
trainnew.columns,testnew.columns,trainnew.shape,testnew.shape

(Index(['id', 'target', 'bank_name_more_cate', 'bank_name_less_cate',
        'card_type_more_cate', 'card_type_less_cate', 'phone_more_cate',
        'phone_less_cate', 'phoneSame', 'havaCreditCard',
        'bank_name_more_count', 'bank_name_less_count', 'card_type_more_count',
        'card_type_less_count', 'phone_more_count', 'phone_less_count',
        'phone_num'],
       dtype='object'),
 Index(['id', 'bank_name_more_cate', 'bank_name_less_cate',
        'card_type_more_cate', 'card_type_less_cate', 'phone_more_cate',
        'phone_less_cate', 'phoneSame', 'havaCreditCard',
        'bank_name_more_count', 'bank_name_less_count', 'card_type_more_count',
        'card_type_less_count', 'phone_more_count', 'phone_less_count',
        'phone_num'],
       dtype='object'),
 (120929, 17),
 (47031, 16))

In [7]:
#'bank_name', 'tail_num', 'card_type', 'phone', 'id'], dtype='object')

trainnew.to_csv("../dealinput/train_bankcard.csv",index = False)
testnew.to_csv("../dealinput/test_bankcard.csv",index = False)
trainnew.head(),testnew.head()

(                   id  target bank_name_more_cate bank_name_less_cate  \
 0  501951980776722440       0                   a                   a   
 1  525890212484616200       0                   c                   c   
 2  599309364691472392       0                   h                   l   
 3  582559913071350024       0                   m                   m   
 4  599017959926534408       0                   e                   e   
 
    card_type_more_cate  card_type_less_cate phone_more_cate phone_less_cate  \
 0                  1.0                  1.0     132****2008     132****2008   
 1                  1.0                  1.0     187****2500     187****2500   
 2                  2.0                  1.0     135****3522     135****3522   
 3                  1.0                  1.0     136****8651     136****8651   
 4                  1.0                  1.0     137****4241     137****4241   
 
    phoneSame  havaCreditCard  bank_name_more_count  bank_name_less_coun

In [8]:
trainnew.shape,testnew.shape

((120929, 17), (47031, 16))